In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime

class BudgetSimulatorTreeviewApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Budget Simulator with Treeview")
        self.root.config(bg="white")

        self.monthly_budget = 0
        self.categories = {}
        self.spent_per_category = {}
        self.transactions = []

        self.setup_ui()

    def setup_ui(self):
        #controls
        frame_top = tk.Frame(self.root)
        frame_top.pack(pady=10)

        tk.Label(frame_top, text="Monthly Budget:").grid(row=0, column=0, padx=5)
        self.budget_entry = tk.Entry(frame_top)
        self.budget_entry.grid(row=0, column=1, padx=5)
        tk.Button(frame_top, text="Set Budget", command=self.set_budget).grid(row=0, column=2, padx=5)

        tk.Label(frame_top, text="Category:").grid(row=1, column=0)
        self.category_entry = tk.Entry(frame_top)
        self.category_entry.grid(row=1, column=1)

        tk.Label(frame_top, text="Amount:").grid(row=1, column=2)
        self.category_amount_entry = tk.Entry(frame_top)
        self.category_amount_entry.grid(row=1, column=3)

        tk.Button(frame_top, text="Add Category", command=self.add_category).grid(row=1, column=4)

        #transaction logging
        frame_log = tk.Frame(self.root)
        frame_log.pack(pady=10)

        tk.Label(frame_log, text="Date (YYYY-MM-DD):").grid(row=0, column=0)
        self.date_entry = tk.Entry(frame_log, width=12)
        self.date_entry.grid(row=0, column=1)

        tk.Label(frame_log, text="Amount:").grid(row=0, column=2)
        self.amount_entry = tk.Entry(frame_log, width=10)
        self.amount_entry.grid(row=0, column=3)

        tk.Label(frame_log, text="Category:").grid(row=0, column=4)
        self.trans_category_entry = tk.Entry(frame_log, width=12)
        self.trans_category_entry.grid(row=0, column=5)

        tk.Label(frame_log, text="Type:").grid(row=0, column=6)
        self.trans_type_var = tk.StringVar(value="Expense")
        tk.OptionMenu(frame_log, self.trans_type_var, "Expense", "Income").grid(row=0, column=7)

        tk.Label(frame_log, text="Description:").grid(row=0, column=8)
        self.description_entry = tk.Entry(frame_log, width=20)
        self.description_entry.grid(row=0, column=9)

        button = tk.Button(frame_log, text="Log Transaction", command=self.log_transaction)
        button.grid(row=0, column=10, padx=10)

        #treeview for transactions
        tk.Label(self.root, text="📄 Transactions").pack()

        self.trans_tree = ttk.Treeview(
            self.root,
            columns=("Date", "Type", "Amount", "Category", "Description"),
            show="headings",
            height=7
        )

        for col in self.trans_tree["columns"]:
            self.trans_tree.heading(col, text=col)
            self.trans_tree.column(col, anchor="center")

        self.trans_tree.pack(pady=5)

        #treeview for category overview
        tk.Label(self.root, text="📊 Category Status").pack()

        self.cat_tree = ttk.Treeview(
            self.root,
            columns=("Budget", "Spent", "Remaining"),
            show="tree headings",
            height=5
        )

        self.cat_tree.heading("#0", text="Category")

        for col in self.cat_tree["columns"]:
            self.cat_tree.heading(col, text=col)
            self.cat_tree.column(col, anchor="center")

        self.cat_tree.pack(pady=5)

        #budget summary
        self.status_label = tk.Label(self.root, text="Unallocated Funds: ₦0.00", fg="blue")
        self.status_label.pack(pady=5)

    #functions

    def set_budget(self):
        try:
            self.monthly_budget = float(self.budget_entry.get())
            self.budget_entry.delete(0, tk.END)
            self.output_status()
            messagebox.showinfo("Success", f"Monthly budget set to ₦{self.monthly_budget:.2f}")
        except ValueError:
            messagebox.showerror("Error", "Invalid budget amount.")

    def add_category(self):
        category = self.category_entry.get()

        try:
            amount = float(self.category_amount_entry.get())

            if category in self.categories:
                messagebox.showerror("Error", "Category already exists.")
                return

            if self.get_unallocated_funds() < amount:
                messagebox.showwarning("Budget Limit", "Not enough unallocated funds.")
                return

            self.categories[category] = amount
            self.spent_per_category[category] = 0

            self.category_entry.delete(0, tk.END)
            self.category_amount_entry.delete(0, tk.END)

            self.refresh_cat_tree()
            self.output_status()

        except ValueError:
            messagebox.showerror("Error", "Invalid amount.")

    def log_transaction(self):
        try:
            date = datetime.strptime(self.date_entry.get(), "%Y-%m-%d").date()
            amount = float(self.amount_entry.get())
            category = self.trans_category_entry.get()
            t_type = self.trans_type_var.get()
            desc = self.description_entry.get()

            if category not in self.categories:
                messagebox.showerror("Error", "Category not found.")
                return

            if t_type == "Expense":
                self.spent_per_category[category] += amount
                remaining = self.categories[category] - self.spent_per_category[category]

                if remaining < 0:
                    messagebox.showwarning("Overspent", f"You've overspent in '{category}'!")
                elif remaining < 0.1 * self.categories[category]:
                    messagebox.showwarning("Budget Warning", f"You're nearing the limit for '{category}'!")

            elif t_type == "Income":
                self.categories[category] += amount
                self.monthly_budget += amount  # <-- Fix: Add income to total monthly budget

            #store transaction
            self.transactions.append((date, t_type, f"₦{amount:.2f}", category, desc))

            #refresh displays
            self.refresh_trans_tree()
            self.refresh_cat_tree()
            self.output_status()

            #clear inputs
            self.date_entry.delete(0, tk.END)
            self.amount_entry.delete(0, tk.END)
            self.trans_category_entry.delete(0, tk.END)
            self.description_entry.delete(0, tk.END)

        except ValueError:
            messagebox.showerror("Error", "Invalid input. Check date or amount.")

    def refresh_trans_tree(self):
        self.trans_tree.delete(*self.trans_tree.get_children())

        for txn in self.transactions:
            self.trans_tree.insert("", "end", values=txn)

    def refresh_cat_tree(self):
        self.cat_tree.delete(*self.cat_tree.get_children())

        for cat in self.categories:
            budget = self.categories[cat]
            spent = self.spent_per_category.get(cat, 0)
            remaining = budget - spent

            self.cat_tree.insert(
                "",
                "end",
                text=cat,
                values=(
                    f"₦{budget:.2f}",
                    f"₦{spent:.2f}",
                    f"₦{remaining:.2f}"
                )
            )

    def get_unallocated_funds(self):
        total_allocated = sum(self.categories.values())
        return self.monthly_budget - total_allocated

    def output_status(self):
        unallocated = self.get_unallocated_funds()
        self.status_label.config(text=f"Unallocated Funds: ₦{unallocated:.2f}")


#run app
root = tk.Tk()
app = BudgetSimulatorTreeviewApp(root)
root.mainloop()


BUDGET SIMULATOR 
The program allows a user to set a monthly budget and allocate it to categories
Transactions (income/expense) are logged per category using a treeview table

Expenses reduce the remaining balance of a category
Income increases both the selected category budget and the total monthly budget

Unallocated funds = monthly budget – total allocated category budgets
The interface updates dynamically after every action